<a href="https://colab.research.google.com/github/CRM48/AIML425-A1/blob/main/Assignmnt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import sys
import numpy as np
import plotly.graph_objects as go
import torch
from torch import nn

# Variables
seed = 1
num_inputs = 10000
num_epochs = 1000

train_percent = 0.7
validation_percent = 0.15
test_percent = 0.15

# Check ratio validity
if ((train_percent + validation_percent + test_percent) != 1.0):
    print("Ratio error in data splitting")
    sys.exit(1)

# Generate Data
np.random.seed(seed)
x = np.random.normal(0, 1, (num_inputs, 3))
norms = np.sqrt(np.sum(np.square(x), axis=1, keepdims=True))
y = x / norms

# Split Data
num_samples = x.shape[0]
num_train = int(train_percent * num_samples)
num_validation = int(validation_percent * num_samples)
num_test = int(test_percent * num_samples)

x_train = x[:num_train]
y_train = y[:num_train]

x_validation = x[num_train:num_train+num_validation]
y_validation = y[num_train:num_train+num_validation]

x_test = x[num_train+num_validation:]
y_test = y[num_train+num_validation:]

# 3D Plot
fig = go.Figure()

# Add training data output
fig.add_trace(go.Scatter3d(
    x=y_train[:, 0], y=y_train[:, 1], z=y_train[:, 2],
    mode='markers',
    marker=dict(size=3, color='blue'),
    name='Normalized y'
))

fig.update_layout(
    title="Input data normalised so length from origin = 1",
    scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z', aspectmode='data')
)

#fig.show()

# The Model
class NNModel(nn.Module):
  def __init__(self):
     super(NNModel).__init__()
     self.net = nn.Sequential(
         nn.Linear(3, 20),
         nn.ReLU(),
         nn.Linear(20, 20),
         nn.ReLU(),
         nn.Linear(20, 3)
     )

  def forward(self, x):
    return self.net(x)




10000
